# Exercise: GPU Temperature Gradient Analysis

## Background
When monitoring GPU performance, understanding how temperature changes across cores over time can help predict thermal throttling and optimize workload distribution. The Exponential Moving Temperature Gradient (EMTG) helps identify areas of the GPU that are heating up too quickly.

## The Problem
You need to implement an algorithm that:
1. Calculates the rate of temperature change between consecutive readings
2. Applies exponential smoothing to reduce noise
3. Identifies potential thermal hotspots

The EMTG formula for each core is:

$$
EMTG_t = \alpha(T_t - T_{t-1}) + (1-\alpha)EMTG_{t-1}
$$

where:
- $T_t$ is the current temperature
- $T_{t-1}$ is the previous temperature
- $\alpha$ is the smoothing factor (0 < α < 1)
- $EMTG_{t-1}$ is the previous gradient value

## Starting Code

In [1]:
#Specifying path to where nvcc exists so that the jupyter notebook reads from it. nvcc is the nvidia cuda compiler for executing cuda. 
import os
os.environ['PATH'] = "/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.6.1-cf4xlcbcfpwchqwo5bktxyhjagryzcx6/bin:" + os.environ['PATH']

In [2]:
%%writefile codes/temp_gradient_analysis.cu
#include <thrust/universal_vector.h>
#include <thrust/transform.h>
#include <thrust/sort.h>
#include <thrust/execution_policy.h>
#include <cstdio>

struct CoreReading {
    float current_temp;    // Current temperature
    float previous_temp;   // Temperature from last reading
    float previous_emtg;   // Previous gradient value
    int core_id;          // Core identifier
};

struct EMTGResult {
    float gradient;
    int core_id;
};

// TODO: Implement this function
thrust::universal_vector<EMTGResult> calculate_emtg(
    const thrust::universal_vector<CoreReading>& readings,
    float alpha = 0.3f  // Smoothing factor
) {
    // Create results vector
    thrust::universal_vector<EMTGResult> results(readings.size());
    
    // TODO: Transform readings into EMTG results
    
    return results;
}

int main() {
    // Example temperature readings for 4 GPU cores
    thrust::universal_vector<CoreReading> readings{
        {85.0f, 80.0f, 0.0f, 0},  // Core 0
        {82.0f, 79.0f, 0.0f, 1},  // Core 1
        {88.0f, 81.0f, 0.0f, 2},  // Core 2
        {83.0f, 80.0f, 0.0f, 3}   // Core 3
    };
    
    // Calculate EMTG for all cores
    auto results = calculate_emtg(readings);
    
    // Find core with highest gradient
    auto it = thrust::max_element(
        thrust::device,
        results.begin(), 
        results.end(),
        [] __device__ (const EMTGResult& a, const EMTGResult& b) {
            return a.gradient < b.gradient;
        }
    );
    
    // Get index of max element
    size_t max_idx = it - results.begin();
    
    // Print results
    printf("Core %d has highest gradient: %.2f°C/s\n", 
           results[max_idx].core_id, 
           results[max_idx].gradient);
}

Overwriting codes/temp_gradient_analysis.cu


In [3]:
%%bash
nvcc -o codes/temp_gradient_analysis --extended-lambda codes/temp_gradient_analysis.cu
./codes/temp_gradient_analysis

Core 0 has highest gradient: 0.00°C/s


## Your Task

1. Implement the `calculate_emtg` function using Thrust's transform iterators.

2. Your implementation should:
   - Compute the exponential moving temperature gradient for each core
   - Track core IDs alongside their gradients
   - Handle edge cases (like first readings) appropriately
   - Use parallel operations where possible

## Tips

1. **Transform Iterator Pattern**
   ```cpp
   auto gradient_iter = thrust::make_transform_iterator(
       readings.begin(),
       [] __device__ (const CoreReading& reading) {
           // Your gradient calculation here
       }
   );
   ```

2. **Smoothing Implementation**
   - Consider how to properly initialize EMTG values for the first reading
   - Think about proper alpha values for different sampling rates

3. **Performance Considerations**
   - Try to minimize memory transfers
   - Consider using structured bindings for cleaner code
   - Think about how to handle multiple iterations efficiently

## Expected Output Format

```
Core Gradients:
Core 0: 1.23°C/s
Core 1: 0.89°C/s
Core 2: 2.15°C/s [HOTSPOT]
Core 3: 0.95°C/s

Potential thermal throttling predicted in: Core 2
```

## Hints

<details>
<summary>🔍 Click for hints if you're stuck</summary>

1. **Getting Started**
   - Break down the EMTG calculation into steps:
     ```cpp
     // Step 1: Calculate basic temperature difference (T_t - T_{t-1})
     // Step 2: Apply exponential smoothing with previous EMTG
     // Step 3: Store result with core ID
     ```

2. **Transform Iterator Tips**
   - Remember to capture alpha in your lambda: `[alpha]`
   - Structure your transform to return EMTGResult
   - Consider using multiple transforms if it makes the code clearer

3. **Common Pitfalls**
   - Don't forget to handle the first reading specially (when previous_emtg is 0)
   - Make sure your comparisons account for floating point precision
   - Remember that thrust::transform_iterator is non-modifying

</details>

## Solution

<details>
<summary>💡 Click to see solution (try solving it yourself first!)</summary>

```cpp
thrust::device_vector<EMTGResult> calculate_emtg(
    const thrust::device_vector<CoreReading>& readings,
    float alpha
) {
    // Create result vector
    thrust::device_vector<EMTGResult> results(readings.size());
    
    // Create transform iterator for EMTG calculation
    auto emtg_iter = thrust::make_transform_iterator(
        readings.begin(),
        [alpha] __device__ (const CoreReading& reading) {
            // Calculate temperature difference
            float temp_diff = reading.current_temp - reading.previous_temp;
            
            // Calculate new EMTG with exponential smoothing
            float new_emtg = alpha * temp_diff + 
                            (1.0f - alpha) * reading.previous_emtg;
            
            // Return result with core ID
            return EMTGResult{new_emtg, reading.core_id};
        }
    );
    
    // Copy results
    thrust::copy(
        emtg_iter,
        emtg_iter + readings.size(),
        results.begin()
    );
    
    return results;
}
```

### Key Points in Solution

1. **Transform Iterator Usage**
   - Creates a single transform that computes the complete EMTG
   - Captures alpha value in the lambda
   - Returns structured EMTGResult with both gradient and core ID

2. **Smoothing Implementation**
   - Correctly applies exponential smoothing formula
   - Uses previous EMTG value for continuity
   - Handles first-time cases through previous_emtg initialization

3. **Memory Efficiency**
   - Single pass through the data
   - No temporary storage needed
   - Minimal memory transfers

</details>

## Challenge Extensions

1. **Advanced Features**
   - Add temporal anomaly detection
   - Implement moving window analysis
   - Add predictive thermal throttling warnings

2. **Optimization Goals**
   - Minimize global memory access
   - Maximize occupancy
   - Reduce warp divergence

